In [1]:
#importa as bibliotecas
import requests
from lxml import html
from numpy import nan, nanmin, all, isnan

In [2]:
# confere o menor numero que nao seja nulo
def clean_nanmin(s):
    if all(isnan(s)):
        return nan
    return nanmin(s)

In [3]:
# lista as possibilidades de siglas http que podem aparecer
def input_urls(find_site):
    find_list = ['https://', 'http://', 'https://www.', 'http://www.']
    find_list = [x+find_site for x in find_list]
    return find_list

In [4]:
# faz a requisicao no google e extrai o html com as urls da pesquisa
def urls_extract(_search, _page=0):
    search_t = _search.replace(' ', '+')
    n_sites = 10
    url = f'https://www.google.com/search?q={search_t}&start={n_sites*_page}'
    page = requests.get(url)
    tree = html.fromstring(page.content)
    only_url = tree.xpath('//a')
    url_pages = [dict(x.items())['href'] for x in only_url]
    url_pages = [x for x in url_pages if x[0:7] == '/url?q=']
    return url_pages

In [5]:
# valida se a url da pesquisa bate com o site do cliente e o menor indice que deu match
def first_index(_urls, _google_return):
    index_list = []
    for find_list_value in _urls:
        find_list_value_slice = [x[7:7+len(find_list_value)] for x in _google_return]
        if find_list_value in find_list_value_slice:
            index_num = find_list_value_slice.index(find_list_value)
            index_num += 1
        else:
            index_num = nan
        index_list.append(index_num)
    min_index = clean_nanmin(index_list)
    min_index = int(min_index) if min_index > 0 else min_index
    return min_index

In [6]:
# faz a busca paginado ate encontrar o site do cliente
def reload(domain, store_name):
    urls = input_urls(domain)
    n_index = nan
    google_page = 0
    for google_page in range(5):
        google_return = urls_extract(store_name, google_page)
        n_index = first_index(urls, google_return)
        if n_index > 0:
            break
    n_index = n_index + google_page*10 if n_index > 0 else nan
    google_page = google_page + 1 if n_index > 0 else nan

    if n_index > 0:
        print(f'[SUCCESS] Your index domain on Google is : {n_index} (page: {google_page})')
    else:
        print('[ERROR] Domain not found on Google ')

Exemplo de cliente no segmento tech:

In [9]:
#primeiro teste de um site conhecido pelo desenvolvedor para homologacao
#o site do cliente nao foi encontrado pesquisando por palavras genericas
reload(domain='draftbr.com', store_name='Futebol Americano')

[ERROR] Domain not found on Google 


In [11]:
# fazendo uma busca por palavras mais coerentes do site do cliente
reload(domain='draftbr.com', store_name='Draftbr futebol americano')

[SUCCESS] Your index domain on Google is : 4 (pagina: 1)


In [8]:
#pesquisando pelo nome completo do site do cliente
#encontrado no index 3 da primeira pagina
#portanto pode ser realizado um trabalho melhor para aumentar o index do cliente
reload(domain='draftbr.com', store_name='Plataforma Draftbr Futebol Americano')

[SUCCESS] Your index domain on Google is : 3 (pagina: 1)


Exemplo de cliente no ramo do varejo:

In [14]:
reload(domain='lojasestrela.com.br', store_name='Loja estrela')

[SUCCESS] Your index domain on Google is : 12 (pagina: 1)


In [15]:
reload(domain='lojasestrela.com.br', store_name='Loja estrela Joanopolis SP')

[SUCCESS] Your index domain on Google is : 3 (pagina: 1)
